In [1]:
# Import Dependencies
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the Provisional CSV

In [2]:
df = pd.read_csv("Training.csv")

# Split the Data into Training and Testing

In [3]:
# Create our features
X = df.drop(columns = ["2018"])

# Create dummy variables
X = pd.get_dummies(X)

# Create our target
y = df[["2018"]]

In [4]:
# Check the balance of our target values
y["2018"].value_counts()

2725    1
1829    1
1253    1
1624    1
7994    1
5238    1
9589    1
3698    1
3308    1
1659    1
2287    1
7512    1
3244    1
4583    1
7014    1
7780    1
1540    1
5381    1
4093    1
2589    1
2524    1
5464    1
7190    1
1938    1
9973    1
2085    1
6795    1
4061    1
6887    1
4293    1
3133    1
Name: 2018, dtype: int64

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

# Balanced Random Forest Classifier

In [6]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
rand_for_class = BalancedRandomForestClassifier(n_estimators = 100, random_state = 1)
rand_for_class.fit(X_train, y_train)

C:\Users\jorda\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


BalancedRandomForestClassifier(random_state=1)

In [7]:
# Calculated the balanced accuracy score
y_prediction = rand_for_class.predict(X_test)
balanced_accuracy_score(y_test, y_prediction)

C:\Users\jorda\anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1850: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


0.0

In [8]:
# Display the confusion matrix
confusion_matrix(y_test, y_prediction)

array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [9]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_prediction))

                   pre       rec       spe        f1       geo       iba       sup

       1540       0.00      0.00      1.00      0.00      0.00      0.00       1.0
       1659       0.00      0.00      0.88      0.00      0.00      0.00       0.0
       2085       0.00      0.00      1.00      0.00      0.00      0.00       1.0
       2287       0.00      0.00      1.00      0.00      0.00      0.00       1.0
       2524       0.00      0.00      0.75      0.00      0.00      0.00       0.0
       2725       0.00      0.00      1.00      0.00      0.00      0.00       1.0
       3133       0.00      0.00      0.88      0.00      0.00      0.00       0.0
       3308       0.00      0.00      0.88      0.00      0.00      0.00       0.0
       3698       0.00      0.00      1.00      0.00      0.00      0.00       1.0
       4093       0.00      0.00      1.00      0.00      0.00      0.00       1.0
       4293       0.00      0.00      0.88      0.00      0.00      0.00       0.0
   

C:\Users\jorda\anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jorda\anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jorda\anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Sensitivity is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
# List the features sorted in descending order by feature importance
features = rand_for_class.feature_importances_
sort_features = np.argsort(features)[::-1]

for i in sort_features:
    print("{}: ({})".format(X.columns.values[i], features[i]))

Unnamed: 0: (0.2103969495872969)
2016: (0.20528639766624182)
2015: (0.19655405326475917)
2014: (0.19562091518798108)
2017: (0.1921416842937209)


# Easy Ensemble AdaBoost Classifier

In [11]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
easy_ens_class = EasyEnsembleClassifier(n_estimators = 100, random_state = 1)
easy_ens_class.fit(X_train, y_train)

C:\Users\jorda\anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [12]:
# Calculated the balanced accuracy score
y_prediction = easy_ens_class.predict(X_test)
balanced_accuracy_score(y_test, y_prediction)

C:\Users\jorda\anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1850: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


0.0

In [13]:
# Display the confusion matrix
confusion_matrix(y_test, y_prediction)

array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [14]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_prediction))

                   pre       rec       spe        f1       geo       iba       sup

       1540       0.00      0.00      1.00      0.00      0.00      0.00       1.0
       1659       0.00      0.00      0.88      0.00      0.00      0.00       0.0
       2085       0.00      0.00      1.00      0.00      0.00      0.00       1.0
       2287       0.00      0.00      1.00      0.00      0.00      0.00       1.0
       2524       0.00      0.00      0.88      0.00      0.00      0.00       0.0
       2725       0.00      0.00      1.00      0.00      0.00      0.00       1.0
       3133       0.00      0.00      0.75      0.00      0.00      0.00       0.0
       3698       0.00      0.00      1.00      0.00      0.00      0.00       1.0
       4093       0.00      0.00      1.00      0.00      0.00      0.00       1.0
       4293       0.00      0.00      0.75      0.00      0.00      0.00       0.0
       4583       0.00      0.00      0.88      0.00      0.00      0.00       0.0
   

C:\Users\jorda\anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jorda\anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jorda\anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Sensitivity is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
